In [5]:
import pandas as pd

df = pd.read_csv("mmc2.csv", index_col=0)

cell_types_to_keep = [
    "preT.DN1.Th", "preT.DN2a.Th", "preT.DN2b.Th", "preT.DN3.Th",
    "T.DN4.Th", "T.ISP.Th", "T.DP.Th", "T.4.Th", "T.8.Th",
    "T.4.Nve.Sp", "T.4.Nve.Fem.Sp", "T.4.Sp.aCD3+CD40.18hr", "T.8.Nve.Sp",
    "Treg.4.25hi.Sp", "Treg.4.FP3+.Nrplo.Co",
    "T8.TN.P14.Sp", "T8.TE.LCMV.d7.Sp", "T8.MP.LCMV.d7.Sp", "T8.IEL.LCMV.d7.Gut",
    "T8.Tcm.LCMV.d180.Sp", "T8.Tem.LCMV.d180.Sp",
    "NKT.Sp", "NKT.Sp.LPS.3hr", "NKT.Sp.LPS.18hr", "NKT.Sp.LPS.3d",
    "LTHSC.34-.BM", "LTHSC.34+.BM", "STHSC.150-.BM", "MPP4.135+.BM"
]

filtered_df = df[cell_types_to_keep]
filtered_df.to_csv("filtered_abT_Tact_Stem.csv")
print("✅ Filtered shape:", filtered_df.shape)

✅ Filtered shape: (17535, 29)


In [6]:
filtered_df.head()

,preT.DN1.Th,preT.DN2a.Th,preT.DN2b.Th,preT.DN3.Th,T.DN4.Th,T.ISP.Th,T.DP.Th,T.4.Th,T.8.Th,T.4.Nve.Sp,...,T8.Tcm.LCMV.d180.Sp,T8.Tem.LCMV.d180.Sp,NKT.Sp,NKT.Sp.LPS.3hr,NKT.Sp.LPS.18hr,NKT.Sp.LPS.3d,LTHSC.34-.BM,LTHSC.34+.BM,STHSC.150-.BM,MPP4.135+.BM
0610005C13Rik,1.022363,1.389747,1.024819,1.024482,1.026430,1.026217,3.010920,1.024462,1.024819,2.726341,...,1.025833,1.024819,1.385805,1.025833,1.575395,1.024819,1.096732,1.096732,1.021750,1.021812
0610007P14Rik,162.641117,206.945221,209.187788,198.421365,215.056475,225.565360,73.904647,138.841383,139.863904,168.924363,...,206.241084,205.309922,165.690720,133.234920,127.894194,195.147548,206.053987,246.105317,192.424636,204.298358
0610009B22Rik,68.070719,82.468806,89.769337,57.661619,76.399214,84.671456,32.828651,27.207241,36.169759,32.753248,...,36.105700,34.348965,25.168975,33.305724,29.284365,33.322384,78.272059,78.837030,68.844751,76.418169
0610009L18Rik,15.450717,13.573968,14.427620,8.249482,1.683173,4.001953,5.595954,6.367369,6.505833,8.262234,...,8.645607,7.268431,3.840215,1.025833,6.283540,12.791348,8.577159,16.791386,15.511549,16.947354
0610009O20Rik,160.246297,125.475307,155.928005,120.692893,118.433597,149.630866,92.040668,76.781112,87.529814,86.523573,...,87.608325,56.128251,109.175415,91.992319,102.035627,108.414405,168.645852,157.926022,155.941641,186.261464


In [7]:
import pandas as pd

# filtered_df is your 17535 × 29 expression matrix (genes × cell types)
transposed = filtered_df.T

print("Transposed shape:", transposed.shape)
transposed.head()


Transposed shape: (29, 17535)


,0610005C13Rik,0610007P14Rik,0610009B22Rik,0610009L18Rik,0610009O20Rik,0610010F05Rik,0610010K14Rik,0610011F06Rik,0610012G03Rik,0610030E20Rik,...,Zwint,Zxda,Zxdb,Zxdc,Zyg11b,Zyx,Zzef1,Zzz3,a,l7Rn6
preT.DN1.Th,1.022363,162.641117,68.070719,15.450717,160.246297,155.767862,146.959609,108.511521,147.315383,110.833671,...,412.930595,18.257996,13.886397,28.773092,174.486586,1102.093601,145.971474,305.759992,1.022363,165.105875
preT.DN2a.Th,1.389747,206.945221,82.468806,13.573968,125.475307,273.998186,166.023337,110.892076,146.009167,137.259987,...,440.133238,17.155614,9.919895,32.955185,133.170236,727.291384,154.743640,260.956800,1.024462,211.656296
preT.DN2b.Th,1.024819,209.187788,89.769337,14.427620,155.928005,154.269870,245.243593,71.760450,161.827642,124.749368,...,364.562447,20.076978,9.562264,31.644284,166.528710,399.395780,132.590213,249.726900,1.374003,202.104434
preT.DN3.Th,1.024482,198.421365,57.661619,8.249482,120.692893,129.834226,225.662176,78.986667,187.582633,178.268286,...,384.143310,15.261224,8.786195,32.263212,164.850226,248.382338,150.298107,291.366650,2.043970,185.255146
T.DN4.Th,1.026430,215.056475,76.399214,1.683173,118.433597,197.843435,252.196726,99.422042,153.613650,152.467579,...,362.089698,12.951121,9.763669,29.455192,146.115393,142.239852,132.912133,274.503519,1.497412,242.887242


In [8]:
from sklearn.decomposition import PCA
import pandas as pd

# Run PCA on the transposed matrix (cell types × genes)
subset = transposed  # already: cell types as rows, gene expression as columns
pca = PCA(n_components=2)
pca_result = pca.fit_transform(subset)

# Store in a DataFrame
pca_df = pd.DataFrame(pca_result, columns=["PC1", "PC2"], index=subset.index)

# Show the result
print(pca_df.head())


                       PC1          PC2
preT.DN1.Th  -21743.135557  3699.921561
preT.DN2a.Th -24496.018064 -1120.650958
preT.DN2b.Th -25021.218271 -7659.251974
preT.DN3.Th  -27946.889142 -8107.314968
T.DN4.Th     -29665.435436 -9460.248192


In [12]:
from sklearn.feature_selection import VarianceThreshold

# Filter out genes (columns in transposed) with near-zero variance across cell types
selector = VarianceThreshold(threshold=0.5)  # you can tune this
filtered_transposed = selector.fit_transform(transposed)

# Keep gene names (columns) after filtering
selected_genes = transposed.columns[selector.get_support()]
filtered_transposed = pd.DataFrame(filtered_transposed, index=transposed.index, columns=selected_genes)


In [13]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
transposed_scaled = scaler.fit_transform(filtered_transposed)

# Now ready for PCA / UMAP


In [15]:
from sklearn.decomposition import PCA
import pandas as pd

# Run PCA
pca = PCA(n_components=2)
pca_result = pca.fit_transform(transposed_scaled)

# Turn it into a DataFrame with correct cell type labels
pca_df = pd.DataFrame(pca_result, columns=["PC1", "PC2"], index=filtered_transposed.index)
